# Calculate original Ikeda for one ship

In [ ]:
# %load ../../imports.py
"""
These is the standard setup for the notebooks.
"""

%matplotlib inline
%load_ext autoreload
%autoreload 2

from jupyterthemes import jtplot
#jtplot.style(theme='onedork', context='notebook', ticks=True, grid=False)

import warnings
warnings.filterwarnings("ignore")
from pylab import rcParams
rcParams['figure.figsize'] = 15, 8


import pandas as pd
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.set_option("display.max_columns", None)
import numpy as np
import os
import matplotlib.pyplot as plt
#plt.style.use('paper')

#import data
import copy
from rolldecay.bis_system import BisSystem
from rolldecay import database
from mdldb.tables import Run

from sklearn.pipeline import Pipeline
from rolldecayestimators.transformers import CutTransformer, LowpassFilterDerivatorTransformer, ScaleFactorTransformer, OffsetTransformer
from rolldecayestimators.direct_estimator_cubic import EstimatorQuadraticB, EstimatorCubic
from rolldecayestimators.ikeda_estimator import IkedaQuadraticEstimator


In [ ]:
db = database.get_db()

df_simplified_ikeda = database.load(rolldecay_table_name='rolldecay_simplified_ikeda', limit_score=0.5, 
                             exclude_table_name='rolldecay_exclude')

df_rolldecay = database.load(rolldecay_table_name='rolldecay_quadratic_b', limit_score=0.9, 
                             exclude_table_name='rolldecay_exclude')

df_rolldecay_cubic = database.load(rolldecay_table_name='rolldecay_cubic_b', limit_score=0.9, 
                             exclude_table_name='rolldecay_exclude')

In [ ]:
df_rolldecay.head()

### Selecting one ship

In [ ]:
project_number=20157375

In [ ]:
mask = (df_rolldecay['project_number'] == project_number)
df_rolldecay.loc[mask]

### Ballast loading condition

In [ ]:
TF=9.4
#TF=11.7
mask = ((df_rolldecay['project_number'] == project_number) &
        (df_rolldecay['TF'] == TF)
       )
df_ballast = df_rolldecay.loc[mask].copy()
df_ballast

In [ ]:
mask = ((df_rolldecay_cubic['project_number'] == project_number) &
        (df_rolldecay_cubic['TF'] == TF)
       )
df_ballast_cubic = df_rolldecay_cubic.loc[mask].copy()
df_ballast_cubic

In [ ]:
df_ballast_simplified_ikeda = df_simplified_ikeda.loc[df_ballast.index].copy()
df_ballast_simplified_ikeda

In [ ]:
meta_data=df_ballast.iloc[0]
meta_data_cubic=df_ballast_cubic.loc[meta_data.name]
meta_data_ikeda=df_ballast_simplified_ikeda.loc[meta_data.name]

db_run = db.session.query(Run).get(int(meta_data.name))
df = database.load_run(db_run)

In [ ]:
lowpass_filter = LowpassFilterDerivatorTransformer(cutoff=2, minimum_score=0.99)
cutter = CutTransformer(phi_max=np.deg2rad(9), phi_min=np.deg2rad(0.25), phi1d_start_tolerance=0.015)
offset_transformer = OffsetTransformer()

steps = [
    ('filter',lowpass_filter),
#    ('scaler',scaler),  # Is froude scaling a good idea??
    ('cutter', cutter), 
    ('offset_transformer',offset_transformer)
]
        
preprocessor = Pipeline(steps) # define the pipeline object.
preprocessor.fit(df)
X = preprocessor.transform(df)

In [ ]:
direct_estimator = EstimatorQuadraticB.load(data=meta_data, X=X)
direct_estimator_cubic = EstimatorCubic.load(data=meta_data_cubic, X=X)
ikeda_estimator = IkedaQuadraticEstimator.load(data=meta_data_ikeda, X=X)

fig,ax=plt.subplots()
direct_estimator.plot_fit(ax=ax)
fig,ax=plt.subplots()
direct_estimator_cubic.plot_fit(ax=ax)
fig,ax=plt.subplots()
ikeda_estimator.plot_fit(ax=ax)

In [ ]:
meta_data_ikeda['B_e']

In [ ]:
meta_data['B_1']

In [ ]:
meta_data['B_2']

## Run Scores2

In [ ]:
from pyscores2.indata import Indata
from pyscores2.runScores2 import Calculation
from pyscores2.output import OutputFile

In [ ]:
indata_file_name = 'data207_name%s_loading_condition_id%s' % (meta_data.name, meta_data.loading_condition_id)
indata_path = os.path.join('scores2', indata_file_name)
indata = Indata()
indata.open(indata_path)

indata.kxx=meta_data['KXX']
indata.displacement=meta_data['Volume']
indata.rho=1000
indata.zcg=meta_data['kg']-meta_data['TA'] #(important)
indata.lcb=meta_data['lcg']

indata.waveDirectionMin=0
indata.waveDirectionMax=180
indata.waveDirectionIncrement=90
indata.speedMin=0
indata.speedMax=18.6
indata.speedIncrement=18.6

#indata.waveFrequenciesMin=
#indata.waveFrequenciesMax=
#indata.waveFrequenciesIncrement=


In [ ]:
outdata_directory=r'scores2/result'
if not os.path.exists(outdata_directory):
    os.mkdir(outdata_directory)

calculation = Calculation(outDataDirectory=outdata_directory)
calculation.run(indata=indata)

In [ ]:
output_file = OutputFile(filePath=calculation.outDataPath)
df = output_file.get_result()

fig,ax=plt.subplots()
for index, group in df.groupby(by=['speed','wave direction'], sort=False):
    group.plot(x='frequencies', y='rollAmplitude', style='o-', label=index, ax=ax)
    
ax.grid(True)
ax.legend();
ax.set_ylabel('Roll');

In [ ]:
df_roll_damping = output_file.get_roll_damping()
df_roll_damping

In [ ]:
df_roll_damping.set_index(['speed','wave_direction'], inplace=True)

In [ ]:
scores_damping = df_roll_damping.loc[0,0]
scores_damping

In [ ]:
BW_scores = scores_damping['calculated_wave_damping_in_roll']

In [ ]:
meta_data['omega0']/np.sqrt(meta_data['scale_factor'])

In [ ]:
import rolldecayestimators.simplified_ikeda as si

In [ ]:
scale_factor = meta_data['scale_factor']
LPP=meta_data['lpp']
Beam=meta_data['beam']
DRAFT=(meta_data['TA']+meta_data['TF'])/2
Volume=meta_data['Volume']
CB=Volume/(Beam*DRAFT*LPP)
CMID=meta_data['A0']
OG=-meta_data['kg'] + DRAFT
PHI=5
lBK=meta_data['BKL']
bBK=meta_data['BKB']
OMEGA=meta_data['omega0']
V=meta_data['ship_speed']*1.852/3.6

#Froude scale:
LPP/=scale_factor
Beam/=scale_factor
DRAFT/=scale_factor
Volume/=scale_factor**3
OG/=scale_factor
lBK/=scale_factor
bBK/=scale_factor
#OMEGA=meta_data['omega0']
V/=np.sqrt(scale_factor)


B44HAT, BFHAT, BWHAT, BEHAT, BBKHAT, BLHAT = si.calculate_roll_damping(LPP=LPP,Beam=Beam,CB=CB,
                                        CMID=CMID,OG=OG,PHI=PHI,lBK=lBK,bBK=bBK,OMEGA=OMEGA,
                                        DRAFT=DRAFT, V=V, verify_input=False, limit_inputs=True)

In [ ]:
si_result = pd.Series(name='SI-method')
si_result['B44HAT']=B44HAT
si_result['BFHAT']=BFHAT
si_result['BWHAT']=BWHAT
si_result['BEHAT']=BEHAT
si_result['BBKHAT']=BBKHAT
si_result['BLHAT']=BLHAT

In [ ]:
si_result

In [ ]:
BWHAT

In [ ]:
from rolldecayestimators import equations
from rolldecayestimators import symbols, lambdas
from rolldecayestimators.substitute_dynamic_symbols import lambdify
import sympy as sp

In [ ]:
lambdas.B44_hat_equation

In [ ]:
lhs=symbols.B_44_hat
rhs=sp.sqrt(symbols.beam/(2*symbols.g))/(symbols.rho*symbols.Disp*symbols.beam**2) 
sp.Eq(lhs, rhs)

In [ ]:
equations.omega_hat_equation

In [ ]:
g=9.81
rho=1000
BW_hat_scores=lambdas.B_hat_lambda(B=BW_scores, Disp=Volume*scale_factor**3, beam=Beam*scale_factor, 
                                   g=g, rho=rho)
BW_hat_scores

In [ ]:
BFHAT+BW_hat_scores+BBKHAT+BLHAT

In [ ]:
PHI=6  # roll amplitude [deg]
B_e = lambdas.B_e_lambda(B_1=df_ballast['B_1'], B_2=df_ballast['B_2'], phi_a=np.deg2rad(PHI), 
                   omega0=df_ballast['omega0'])


df_ballast['B_e_hat'] = lambdas.B_e_hat_lambda(B_e=B_e, Disp=Volume, beam=Beam, 
                                 g=g, rho=rho)
df_ballast[['ship_speed','B_e_hat']]
df_ballast['V']=df_ballast['ship_speed']*1.852/3.6/np.sqrt(scale_factor)

In [ ]:
B44HAT

## Ikeda

In [ ]:
from rolldecayestimators import ikeda_speed

In [ ]:
inputs = pd.DataFrame()
N=50
speeds=np.linspace(df_ballast['ship_speed'].min(),df_ballast['ship_speed'].max(),N)*1.852/3.6/np.sqrt(scale_factor)
inputs['w'] = w = np.linspace(df_ballast['omega0'].min(),df_ballast['omega0'].max(),N)  # roll frequency [rad/s]

inputs['V'] = speeds  # Ship speed [m/s]
inputs['d'] = DRAFT  # Draught of hull [m]
#BW_scores_model=BW_scores/(scale_factor**4/np.sqrt(scale_factor))
#inputs['Bw0'] = BW_scores_model  # Zero speed wave damping [Nm*s/rad]
ws,B_W0 = output_file.calculate_B_W0()
#B_W0/=(scale_factor**4/np.sqrt(scale_factor))
#ws/=np.sqrt(scale_factor)
#ws*=np.sqrt(scale_factor)

B_W0/=(scale_factor**3.5)
B_W0/=100
ws*=np.sqrt(scale_factor)

inputs['Bw0']=np.interp(w,ws,B_W0)  # Zero speed wave damping [Nm*s/rad]
inputs['fi_a'] = np.deg2rad(PHI) # Roll amplitude [rad]
inputs['B'] = Beam # Breadth of hull [m]
inputs['A'] = CMID*Beam*DRAFT # Area of cross section of hull [m2]
inputs['bBK'] = bBK  #breadth of Bilge keel [m] !!(=height???)
inputs['R'] = 0.1 # Bilge Radis [m]
inputs['OG'] = OG # distance from roll axis to still water level [m]
inputs['Ho'] = Beam/(2*DRAFT)  # half breadth to draft ratio B/(2*d) [-]
inputs['ra'] = ra = rho  # density of water (1025) [kg/m3]
inputs['Cb'] =CB  # Block coeff [-]
inputs['L']=LPP  # Ship length [m]
inputs['LBK']=lBK  # Bilge keel length [m]
inputs['Bw_div_Bw0_max'] = np.inf  # maxmum allowed difference between Bw0 and at speed.
inputs['visc']=1.15*10**-6
inputs['g']=g=9.81
results_ikeda = inputs.apply(ikeda_speed.calculate_B44_series,axis=1)

In [ ]:
results_ikeda_hat = results_ikeda.apply(func=lambdas.B_hat_lambda,Disp=Volume, beam=Beam, 
                                   g=g, rho=rho)
results_ikeda_hat = pd.concat((inputs, results_ikeda_hat), axis=1)


In [ ]:
inputs_si = pd.DataFrame()
inputs_si['V']=speeds

inputs_si['LPP']=LPP
inputs_si['Beam']=Beam
inputs_si['DRAFT']=DRAFT
inputs_si['CB']=Volume/(Beam*DRAFT*LPP)
inputs_si['CMID']=CMID
inputs_si['OG']=OG
inputs_si['PHI']=PHI
inputs_si['lBK']=lBK
inputs_si['bBK']=bBK
inputs_si['OMEGA']=inputs['w'] 

In [ ]:
def si_method(row):
    
    B44HAT, BFHAT, BWHAT, BEHAT, BBKHAT, BLHAT = si.calculate_roll_damping(**row, verify_input=False, 
                                                                           limit_inputs=True, 
                                                                           BWHAT_lim=np.inf)
    si_result = pd.Series(name=row.name)
    si_result['B44HAT']=B44HAT
    si_result['BFHAT']=BFHAT
    si_result['BWHAT']=BWHAT
    si_result['BEHAT']=BEHAT
    si_result['BBKHAT']=BBKHAT
    si_result['BLHAT']=BLHAT 
    
    return si_result

In [ ]:
results_si_hat=inputs_si.apply(func=si_method, axis=1)
results_si_hat = pd.concat((inputs_si, results_si_hat), axis=1)

In [ ]:
renamings = {
    'B44HAT':'B_44',
    'BFHAT':'B_F',
    'BWHAT':'B_W',
    'BEHAT':'B_E',
    'BBKHAT':'B_BK',
    'BLHAT':'B_L',
}

results_si_hat.rename(columns=renamings, inplace=True)

<a id='ikeda_vs_si'></a>

In [ ]:
fig,ax=plt.subplots()
results_ikeda_hat.plot.area(x='V', y = ['B_BK','B_F','B_L','B_W',], ax=ax)
df_ballast.plot(x='V', y='B_e_hat', style='ro', label='model test', ax=ax)
ax.legend()
#ax.set_ylim(0,0.014)
ax.set_title('Original Ikeda compared to model tests')

fig,ax=plt.subplots()
results_si_hat.plot.area(x='V', y = ['B_BK','B_F','B_E','B_L','B_W',], ax=ax)
df_ballast.plot(x='V', y='B_e_hat', style='ro', label='model test', ax=ax)
ax.legend()
ax.set_ylim(0,0.014)
ax.set_title('Simplified Ikeda compared to model tests')

In [ ]:
results_ikeda_hat['B_E'] = results_si_hat['B_E'].copy()
results_si_hat['B_BK'] = results_ikeda_hat['B_BK'].copy()


<a id='ikeda_vs_si2'></a>

In [ ]:
fig,ax=plt.subplots()
results_ikeda_hat.plot.area(x='V', y = ['B_BK','B_F','B_E','B_L','B_W',], ax=ax)
df_ballast.plot(x='V', y='B_e_hat', style='ro', label='model test', ax=ax)
ax.legend()
ax.set_ylim(0,0.014)
ax.set_title('Original Ikeda compared to model tests')

fig,ax=plt.subplots()
results_si_hat.plot.area(x='V', y = ['B_BK','B_F','B_E','B_L','B_W',], ax=ax)
df_ballast.plot(x='V', y='B_e_hat', style='ro', label='model test', ax=ax)
ax.legend()
ax.set_ylim(0,0.014)
ax.set_title('Simplified Ikeda compared to model tests')


<a id='lewis_coefficients'></a>
## Sectional Lewis coefficients
$$a=B / 2\left(1+a_{1}+a_{3}\right)$$
$$a_{1}=\left(1+a_{3}\right)(H-1) /(H+1)$$
$$a_{3}=\frac{\left.-C_{1}+3+\sqrt{(} 9-2 C_{1}\right)}{C_{1}}$$
$$C_{1}=\left(3+\frac{4 \sigma_{s}}{\pi}\right)+\left(1-\frac{4 \sigma_{s}}{\pi}\right)\left(\frac{H-1}{H+1}\right)^{2}$$
$$\sigma_{\mathrm{s}}=\frac{S}{B T}$$
$$H=\frac{B}{2 T}$$

In [ ]:
Ts=indata.ts/scale_factor
bwl=indata.bs/scale_factor

areas=indata.cScores*Ts*bwl
a, a_1, a_3, sigma_s, H = ikeda_speed.calculate_sectional_lewis(B_s=bwl, T_s=Ts, S_s=areas)
    

<a id='eddy'></a>

In [ ]:
xs=np.linspace(0,LPP,21)
R=Beam*0.05  # Guessing
d=DRAFT
wE=inputs.iloc[0]['w']
fi_a = inputs.iloc[0]['fi_a']
B_E = ikeda_speed.eddy(bwl=bwl, a_1=a_1, a_3=a_3, sigma=sigma_s, xs=xs, H0=H, Ts=Ts, OG=OG, 
                 R=R, d=d, wE=wE, fi_a=fi_a)

In [ ]:
B_E_hat=lambdas.B_hat_lambda(B_E,Disp=Volume, beam=Beam, 
                                   g=g, rho=rho)
B_E_hat

## Using Ikeda class

In [ ]:
from rolldecayestimators.ikeda import Ikeda, IkedaR
fi_a = inputs['fi_a']
w = inputs['w']
V = inputs['V']
ikeda = Ikeda.load_scoresII(V=V, w=w, fi_a=fi_a, indata=indata, output_file=output_file, 
                            scale_factor=scale_factor, lBK=inputs['LBK'], bBK=inputs['bBK'])

ikeda.R = inputs['R'].min()


In [ ]:
def calculate(inputs, ikeda):

    output = inputs.copy()
    output['B_44_hat'] = ikeda.calculate_B44()
    output['B_W0'] =ikeda.calculate_B_W0()
    output['B_W'] =ikeda.calculate_B_W()
    output['B_F'] =ikeda.calculate_B_F()
    output['B_E'] =ikeda.calculate_B_E()
    output['B_BK'] =ikeda.calculate_B_BK()
    output['B_L'] =ikeda.calculate_B_L()
    output['Bw_div_Bw0'] =ikeda.calculate_Bw_div_Bw0()
    
    return output

<a id="ikeda_r_guess"></a>

In [ ]:
output = calculate(inputs=inputs, ikeda=ikeda)

fig,ax=plt.subplots()
output.plot.area(x='V', y = ['B_BK','B_F','B_E','B_L','B_W',], ax=ax)
ax.legend()
ax.set_ylim(0,0.014)
ax.set_title('Original Ikeda compared to model tests');

In [ ]:
ikeda_r = IkedaR.load_scoresII(V=V, w=w, fi_a=fi_a, indata=indata, output_file=output_file, 
                            scale_factor=scale_factor, lBK=inputs['LBK'], bBK=inputs['bBK'])


<a id="ikeda_r_ikeda"></a>

In [ ]:
output = calculate(inputs=inputs, ikeda=ikeda_r)

fig,ax=plt.subplots()
output.plot.area(x='V', y = ['B_BK','B_F','B_E','B_L','B_W',], ax=ax)
df_ballast.plot(x='V', y='B_e_hat', style='ro', label='model test', ax=ax)
ax.legend()
ax.set_ylim(0,0.014)
ax.set_title('Original Ikeda compared to model tests');

In [ ]:
def calculate_one(ikeda):

    output = pd.Series()
    output['B_44_hat'] = ikeda.calculate_B44()[0]
    output['B_W0'] =ikeda.calculate_B_W0()[0]
    output['B_W'] =ikeda.calculate_B_W()[0]
    output['B_F'] =ikeda.calculate_B_F()[0]
    output['B_E'] =ikeda.calculate_B_E()[0]
    output['B_BK'] =ikeda.calculate_B_BK()[0]
    output['B_L'] =ikeda.calculate_B_L()[0]
    output['Bw_div_Bw0'] =ikeda.calculate_Bw_div_Bw0()[0]
    
    return output

<a id="R_dependency"></a>

In [ ]:
from rolldecayestimators.ikeda import Ikeda, IkedaR
fi_a = inputs['fi_a'].mean()
w = inputs['w'].mean()
V = 0
Rs = np.linspace(0.01,0.2,50)*inputs.iloc[0]['B']
ikeda_R = Ikeda.load_scoresII(V=V, w=w, fi_a=fi_a, indata=indata, output_file=output_file, 
                            scale_factor=scale_factor, lBK=inputs['LBK'].iloc[0], 
                              bBK=inputs['bBK'].iloc[0])

outputs=pd.DataFrame()
for R_ in Rs:
    ikeda_R.R=R_
    output = calculate_one(ikeda=ikeda_R)
    outputs=outputs.append(output, ignore_index=True)
    
fig,ax=plt.subplots()
outputs['R']=Rs
outputs[r'R/beam']=Rs/inputs.iloc[0]['B']
outputs.plot.area(x=r'R/beam', y = ['B_BK','B_F','B_E','B_L','B_W',], ax=ax);
